In [ ]:
# Retro to play street fighter, gym to create the environment
import gym, retro

# in roms python -m retro.import .

# import time to slow down the game
import time

# retrowrapper to create multiple environments in parallel execution

# Check versions
print("gym:", gym.__version__)    # should print 0.21.0
print("retro:", retro.__version__) # should print 0.8.0 (or whatever gym-retro version)


In [ ]:
import os
import sys

print(f"Python version: {sys.version}")
print(f"Retro version: {retro.__version__ if hasattr(retro, '__version__') else 'Unknown'}")

In [ ]:
retro.data.list_games()


In [ ]:
print(retro.__version__)
print(gym.__version__)
print(dir(retro))
print(retro.get_romfile_system)

In [ ]:
# Starts up the game environment
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')


In [ ]:
env.observation_space

In [ ]:
env.action_space

# env.action_space.sample()

# Example: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] means only the button "B" is pressed
# 12 posible binary permutations (2^12 = 4096)


In [ ]:
# Random agent that plays the game

# Reset game to initial state
obs = env.reset()

done = False
iterations = 1 #Modify to change number of games played

for game in range(iterations):
    while not done:
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        # time.sleep(0.01) # Uncomment to slow down the game
        print(reward, info, done)
    obs = env.reset()   
    done = False

In [ ]:
# Parameters to consider
info

In [ ]:
# Setup Enviroment 

# Observation Preprocess - gray scale, frame delta, resize
# Filter actions - parameter
# Change Reward function 


# Import environment base class for a wrapper
from gym import Env
# Import the space shapes for the environment
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta
import numpy as np
# Import cv2 for gray scale 
import cv2
# pip install opencv-python

# Import matplotlib to plot images
from matplotlib import pyplot as plt
# pip install matplotlib

In [ ]:
gray = cv2.cvtColor(obs, cv2.COLOR_BGR2GRAY)
resize = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)

plt.imshow(cv2.cvtColor(resize, cv2.COLOR_BGR2RGB))

In [ ]:
# Create custom environment
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        # Specify action and observation space
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Start and instance the game
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis',
                               use_restricted_actions=retro.Actions.FILTERED)
        
    def reset(self):
        # Return the first frame
        obs = self.game.reset()
        # Frame delta = Current frame - previous frame        
        obs = self.preprocess(obs)

        self.previouse_frame = obs
        # Create an attribute to hold the score delta
        self.score = 0

        return obs

    def preprocess(self, observation):
        # Grayscaling
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resizing
        resize = cv2.resize(gray, (84, 84), interpolation = cv2.INTER_CUBIC)

        # Add the channels value
        channels = np.reshape(resize, (84, 84, 1))

        return channels

    def step(self, action):
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        # Frame delta = Current frame - previous frame
        frame_delta = obs - self.previouse_frame
        self.previouse_frame = obs

        # Custom reward function
        reward = info['score'] - self.score
        self.score = info['score']

        return frame_delta, reward, done, info

    def render(self, *args, **kwargs):
        self.game.render()

    def close(self):
        self.game.close()


In [ ]:
env.close()

In [ ]:
env = StreetFighter()

In [ ]:
# Random agent that plays the game

# Reset game to initial state
obs = env.reset()

done = False
iterations = 5 #Modify to change number of games played

for game in range(iterations):
    while not done:
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        # time.sleep(0.01) # Uncomment to slow down the game
        if reward != 0:
            print(reward, info, done)
    obs = env.reset()   
    done = False